In [12]:
import sys
sys.path.insert(0, '../')

In [13]:
from pprint import pprint
from collections import Counter

import rdflib
import wlkernel
import math
import numpy as np

In [14]:
rdf_graph = rdflib.Graph().parse('../data/aifbfixed_complete.n3', format='n3')

In [15]:
affiliation_uri = rdflib.URIRef('http://swrc.ontoware.org/ontology#affiliation')
instances = [ (t[0], t[2]) for t in rdf_graph.triples( (None, affiliation_uri, None) ) ]
instances = [(x, _) for (x, _) in instances if x != rdflib.term.URIRef('http://www.aifb.uni-karlsruhe.de/Personen/viewPersonOWL/id1959instance')]
assert len(instances) == len(set(instances))
print(len(instances))

177


In [16]:
instances_id = [str(x[0]) for x in instances]
instances_class = [str(x[1]) for x in instances]
# pprint(Counter(instances_class).most_common())

In [17]:
employs_uri = rdflib.term.URIRef('http://swrc.ontoware.org/ontology#employs')
head_uri = rdflib.term.URIRef('http://swrc.ontoware.org/ontology#head')
member_uri = rdflib.term.URIRef('http://swrc.ontoware.org/ontology#member')

#predicates_to_remove = [employs_uri, head_uri, member_uri, affiliation_uri]
predicates_to_remove = [employs_uri, affiliation_uri]

for predicate in predicates_to_remove:
    rdf_graph.remove( (None, predicate, None) )

In [18]:
triples = ((str(s), str(p), str(o)) for s, p, o in rdf_graph)
g = wlkernel.WLRDFGraph(triples=triples, instances=instances_id, max_depth=2)

In [19]:
kernel_matrix = wlkernel.wlrdf_kernel_matrix(graph=g, instances=instances_id, iterations=1)

In [20]:
from sklearn import svm
svm_clf = svm.SVC(kernel='precomputed')
svm_clf.fit(kernel_matrix, instances_class)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='precomputed', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [21]:
y_pred = svm_clf.predict(kernel_matrix)

In [23]:
from sklearn.metrics import accuracy_score
print('Train accuracy: ', accuracy_score(instances_class, y_pred))

Train accuracy:  0.6214689265536724
